In [ ]:
#r "./../../../../../../public/src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
using System;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using System.IO;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution.Gnuplot;
using BoSSS.Solution.AdvancedSolvers;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

Using gnuplot: B:\BoSSS-BIN\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
ArgumentException caught while opening database #2 at 'H:\weber': Database Error: field state directory (H:\weber\timesteps) does not exist.
Databases loaded: 
Capacity: 4
Count: 3



In [ ]:
var DB = OpenOrCreateDatabase(@"W:\work\scratch\jw52xeqa\DB_rotSphere_CoreScaling"); DB.Sessions

Opening existing database 'W:\work\scratch\jw52xeqa\DB_rotSphere_CoreScaling'.


#0: strong_scaling	J17496_k4_t1_Stokes__c8_Re50_mue_1	09/16/2021 12:00:08	c945104d...
#1: test	J12288_k4_t1_Stokes__c256_Re50_mue_1	09/16/2021 13:36:01	332d16fc...
#2: test	J12288_k4_t1_Stokes__c128_Re50_mue_1	09/16/2021 13:33:54	b8cd456b...
#3: strong_scaling	J17496_k4_t1_Stokes__c64_Re50_mue_1	09/16/2021 03:52:00	96ef54d7...
#4: strong_scaling	J65856_k2_t1_Stokes__c8_Re50_mue_1	09/16/2021 03:36:22	2fe968b3...
#5: strong_scaling	J17496_k4_t1_Stokes__c256_Re50_mue_1	09/16/2021 03:36:25	20636b15...
#6: strong_scaling	J17496_k4_t1_Stokes__c32_Re50_mue_1	09/16/2021 02:54:38	04d675c5...
#7: strong_scaling	J31944_k3_t1_Stokes__c64_Re50_mue_1	09/16/2021 03:48:46	868162ce...
#8: strong_scaling	J17496_k4_t1_Stokes__c128_Re50_mue_1	09/16/2021 03:11:38	356b79c5...
#9: strong_scaling	J31944_k3_t1_Stokes__c256_Re50_mue_1	09/16/2021 03:24:25	ebbaeb37...
#10: strong_scaling	J17496_k4_t1_Stokes__c16_Re50_mue_1	09/16/2021 02:35:54	4529908a...
#11: strong_scaling	J65856_k2_t1_Stokes__c256_Re50_mue_1	09

In [ ]:
DB.Sessions.Where(s=>Convert.ToInt32(s.KeysAndQueries["DGdegree:Velocity*"])==3).Where(s=>s.ProjectName=="strong_scaling")

#0: strong_scaling	J31944_k3_t1_Stokes__c64_Re50_mue_1	09/16/2021 03:48:46	868162ce...
#1: strong_scaling	J31944_k3_t1_Stokes__c256_Re50_mue_1	09/16/2021 03:24:25	ebbaeb37...
#2: strong_scaling	J31944_k3_t1_Stokes__c128_Re50_mue_1	09/16/2021 03:11:15	90e424ed...
#3: strong_scaling	J31944_k3_t1_Stokes__c32_Re50_mue_1	09/16/2021 02:48:59	889dd618...
#4: strong_scaling	J31944_k3_t1_Stokes__c16_Re50_mue_1	09/16/2021 02:07:57	031c8f78...
#5: strong_scaling	J31944_k3_t1_Stokes__c8_Re50_mue_1	09/16/2021 01:30:00	8e141e62...


In [ ]:
var SomeSessions = DB.Sessions.Where(s=>s.ProjectName=="strong_scaling");

In [ ]:
static var PreComputeProfiling = new Dictionary<System.Guid,MethodCallRecord[]>();
foreach(var sess in SomeSessions){
    MethodCallRecord[] mcrs = null;
    try{
        mcrs = sess.GetProfiling();
    } catch (Exception ex){
        Console.WriteLine("Name:"+sess.Name);
        Console.WriteLine(ex.Message);
    }
    PreComputeProfiling.Add(sess.ID, mcrs);
}

In [ ]:
static class AddCols {
    static public MethodCallRecord[] GetMCR(ISessionInfo SI){
        MethodCallRecord[] mcrs = null;
        PreComputeProfiling.TryGetValue(SI.ID,out mcrs);
        return mcrs;
    }
    
    static private double TimeOnProc(MethodCallRecord mcr, string method){
        double time = 0;
        foreach(var child in mcr.FindChildren(method)){
            time += child.TimeSpentInMethod.TotalSeconds;
        }
        return time;
    }
    
    static public double AverageTime(ISessionInfo SI,string method){
        var mcrs       = GetMCR(SI);
        double avtime = 0;

        foreach(var mcr in mcrs){
            avtime+=TimeOnProc(mcr,method);
        }
        avtime = avtime / mcrs.Count();
        return avtime;
    }

    static public double MaxTime(ISessionInfo SI,string method){
        var mcrs       = GetMCR(SI);
        double maxtime = -1;

        foreach(var mcr in mcrs){
            double time =TimeOnProc(mcr,method);    
            maxtime = time>maxtime?time:maxtime;
        }
        return maxtime;
    }

    static public object LSolverIter(ISessionInfo SI){
        var mcr          = GetMCR(SI)[0];
        string searchstr = "BoSSS.Solution.AdvancedSolvers.DirectSolver.Solve";
        var ndS = mcr.FindChildren(searchstr);
        var nd  = ndS.ElementAt(0);
        return nd.CallCount;
    }

    static public object NoOfCores(ISessionInfo SI){
        return SI.ComputeNodeNames.Count();
    }

    static public object NoOfNodes(ISessionInfo SI){
        return SI.ComputeNodeNames.GroupBy(s=>s).Count();
    }
    
    static public object MGDepth(ISessionInfo SI){
        var mcr = GetMCR(SI)[0];
        double NInit  = mcr.FindChildren("BoSSS.Solution.AdvancedSolvers.OrthonormalizationMultigrid.Init").ToArray().Length;
        return NInit + 1;
    }
}

In [ ]:
//SomeSessions.Select(s=>AddCols.MGDepth(s))
//SomeSessions.Pick(0).KeysAndQueries.Keys.Contains("DGdegree:Velocity*")
SomeSessions.Select(s=>AddCols.LSolverIter(s))

index,value
0,381
1,139
2,399
3,90
4,312
5,232
6,92
7,171
8,230
9,420


In [ ]:
string[] Functraces = new string[]{
               
               "BoSSS.Solution.AdvancedSolvers.OrthonormalizationMultigrid.AddSol",
               "BoSSS.Solution.AdvancedSolvers.Schwarz.Solve",
               "BoSSS.Solution.AdvancedSolvers.OrthonormalizationMultigrid.MinimizeResidual",
               "BoSSS.Solution.AdvancedSolvers.OrthonormalizationMultigrid.Residual",
               "BoSSS.Solution.AdvancedSolvers.OrthonormalizationMultigrid.Solve",
               "overlap_scaling",
               "block_solve_level",
               "caching",
               "nocaching",
               
               //"Slv Iter"
            };

In [ ]:
var SesTab = new System.Data.DataTable();
var Col         = SesTab.Columns.Add("NoCores", typeof(Int32));  
Col.AllowDBNull = false;
SesTab.Columns.Add("Iter", typeof(Int32)); 
SesTab.Columns.Add("DGdegree", typeof(Int32)); 
var ColDic = new Dictionary<string, string>();
foreach(string Ftrace in Functraces){
    string[] Schnipsel = Ftrace.Split('.');
    int L = Schnipsel.Length;
    string ColumnName = "";
    if(L>1)
        ColumnName = String.Concat(Schnipsel[L-2],'.',Schnipsel[L-1]);
    else
        ColumnName = Schnipsel.Last();
    SesTab.Columns.Add(ColumnName,typeof(Double));
    ColDic.Add(ColumnName,Ftrace);
} SesTab.GetColumnNames()

index,value
0,NoCores
1,Iter
2,DGdegree
3,OrthonormalizationMultigrid.AddSol
4,Schwarz.Solve
5,OrthonormalizationMultigrid.MinimizeResidual
6,OrthonormalizationMultigrid.Residual
7,OrthonormalizationMultigrid.Solve
8,overlap_scaling
9,block_solve_level


In [ ]:
foreach(var sess in SomeSessions){
    var newrow = SesTab.NewRow();
    foreach(var col in SesTab.Columns){
        string cName = (col as System.Data.DataColumn).ColumnName;
        ColDic.TryGetValue(cName, out string Ftrace);
        try{
            newrow[cName]=AddCols.MaxTime(sess,Ftrace);
        } catch (Exception ex){
            //Die anderen Columns halt ...
        } 
    }
    newrow["NoCores"] = Convert.ToInt32(AddCols.NoOfCores(sess));
    newrow["Iter"] = Convert.ToInt32(AddCols.LSolverIter(sess));
    newrow["DGdegree"]=Convert.ToInt32(sess.KeysAndQueries["DGdegree:Velocity*"]);
    SesTab.Rows.Add(newrow);
} SesTab.Print()

    NoCores Iter DGdegree OrthonormalizationMultigrid.AddSol Schwarz.Solve      OrthonormalizationMultigrid.MinimizeResidual OrthonormalizationMultigrid.Residual OrthonormalizationMultigrid.Solve overlap_scaling    block_solve_level  caching nocaching 
0:  8       381  4        2208.1617849                       6202.1171886       3.7931965                                    58.481997899999996                   13191.9226454                     1330.4015579       5372.3734328       0       0         
1:  64      139  4        243.5833553                        1567.9947702       1.5950494000000002                           6.769698                             5264.462673600001                 595.9164357000001  1366.0228619       0       0         
2:  8       399  2        1393.304426                        1409.4460829999998 3.6236107                                    17.9729705                           4117.792163                       129.2219896        1160.0557056999999 0      

In [ ]:
var theDict = new List<Tuple<string,DashTypes,LineColors>>();
public static Tuple<string,DashTypes,LineColors>[] AofFormats;
var dashes = new DashTypes[]{DashTypes.Dashed,DashTypes.DotDashed,DashTypes.DotDotDashed};
var points= new PointTypes[]{PointTypes.OpenCircle,PointTypes.OpenCircle,PointTypes.OpenDiamond,PointTypes.OpenUpperTriangle};
var colors = new LineColors[]{LineColors.Red, LineColors.Green, LineColors.Blue, LineColors.Magenta, LineColors.Yellow, LineColors.Black, LineColors.Orange, LineColors.Grey};

int iSweep=0;
foreach(string Name in SesTab.GetColumnNames()){
    //if(Name!="OrthonormalizationMultigrid.MinimizeResidual"&&Name!="overlap_scaling")
    //    continue;
    if(Name=="NoCores"||Name=="Iter"||Name=="DGdegree")
        continue;
    int one=iSweep%(dashes.Length-1);
    int two=iSweep/(dashes.Length-1);
    theDict.Add(new Tuple<string,DashTypes,LineColors>(Name,dashes[one],colors[two]));
    iSweep++;
}
AofFormats = theDict.ToArray()

index,Item1,Item2,Item3
0,OrthonormalizationMultigrid.AddSol,Dashed,Red
1,Schwarz.Solve,DotDashed,Red
2,OrthonormalizationMultigrid.MinimizeResidual,Dashed,Green
3,OrthonormalizationMultigrid.Residual,DotDashed,Green
4,OrthonormalizationMultigrid.Solve,Dashed,Blue
5,overlap_scaling,DotDashed,Blue
6,block_solve_level,Dashed,Magenta
7,caching,DotDashed,Magenta
8,nocaching,Dashed,Yellow


In [ ]:
public static class DoWhatUWant{
    public static PlotRowSelectorEx Solver_DG_Proj(int DG){
        return delegate (int iSweep, int iTabRow, 
                     IDictionary<string, object> Row, 
                     out string Nmn, 
                     out PlotFormat Fmt,
                     out double xValue, out double yValue) { 

               int k  = Convert.ToInt32(Row["DGdegree"]);

               if(k != DG ) {
                    // degree does not match -> not in this plot
                    Nmn    = null;
                    Fmt    = null;
                    xValue = 0;
                    yValue = 0;
                    return;
               } 
               
               Fmt           = new PlotFormat();
               Fmt.PointSize = 0.5;
               Fmt.Style     = Styles.LinesPoints;
               Fmt.LineWidth = 3;

               Nmn = AofFormats[iSweep].Item1;
               Fmt.DashType = AofFormats[iSweep].Item2;
               Fmt.LineColor = AofFormats[iSweep].Item3;
               double iter = Convert.ToDouble(Row["Iter"]);
               yValue = Convert.ToDouble(Row[Nmn])/iter;
               xValue    = Convert.ToDouble(Row["NoCores"]);
               
           };
    }

    public static PlotRowSelectorEx Solver_DG_Proj(){
        return delegate (int iSweep, int iTabRow, 
                     IDictionary<string, object> Row, 
                     out string Nmn, 
                     out PlotFormat Fmt,
                     out double xValue, out double yValue) { 

               int k           = Convert.ToInt32(Row["DGdegree"]);
               
               Fmt           = new PlotFormat();
               Fmt.PointSize = 0.5;
               Fmt.Style     = Styles.LinesPoints;
               Fmt.LineWidth = 3;
               
               switch(k){
                case 2:
                    Nmn="k2";
                    Fmt.LineColor = LineColors.Red;
                    break;
                case 3:
                    Nmn="k3";
                    Fmt.LineColor = LineColors.Orange;
                    break;
                case 4:
                    Nmn="k4";
                    Fmt.LineColor = LineColors.Blue;
                    break;
                default:
                    throw new Exception("vergiss es");
                }
                double time = Convert.ToDouble(Row["Slv Iter"]);
                double iter = Convert.ToDouble(Row["Iter"]);
                //yValue = time/iter;
                yValue=Convert.ToDouble(Row["Iter"]);
                xValue    = Convert.ToDouble(Row["NoCores"]);
           };
    }

}
    

In [ ]:
int k = 4;
int xMin = 4;
int xMax = 512;

In [ ]:
Plot2Ddata[,] multiplots = new Plot2Ddata[1,1];
multiplots[0,0] = SesTab.ToPlot(DoWhatUWant.Solver_DG_Proj(k),AofFormats.Length);
multiplots[0,0].LogX = true;
multiplots[0,0].LogY = true;
multiplots[0,0].tmargin = 0;
multiplots[0,0].bmargin = 5;
multiplots[0,0].XrangeMin=xMin;
multiplots[0,0].XrangeMax=xMax;
multiplots[0,0].Ylabel = "wallclock time";
multiplots[0,0].Xlabel = "cores";
multiplots[0,0].LegendAlignment=new string[]{"i","b","r"};
multiplots.PlotNow()

Using gnuplot: B:\BoSSS-BIN\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"inside bottom right Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -3 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 wallclock time 
 
 
 
 
 cores 
 
 
 
 
 OrthonormalizationMultigrid.AddSol 
 
 
 
 
 OrthonormalizationMultigrid.AddSol 
 
 
 
 
 
 
 
 
 
 
 
 
 Schwarz.Solve 
 
 
 Schwarz.Solve 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthonormalizationMultigrid.MinimizeResidual 
 
 
 OrthonormalizationMultigrid.MinimizeResidual 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthonormalizationMultigrid.Residual 
 
 
 OrthonormalizationMultigrid.Residual 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthonormalizationMultigrid.Solve 
 
 
 OrthonormalizationMultigrid.Solve 
 
 
 
 
 
 
 
 
 
 
 
 
 overlapscaling 
 
 
 overlap s caling 
 
 
 
 
 
 
 
 
 
 
 
 
 blocksolvelevel 
 
 
 block s olve l evel 
 
 
 
 
 
 
 
 
 
 
 
 
 caching 
 
 
 caching 
 
 
 
 
 
 
 nocaching 
 
 
 nocaching